# Merges Cleaned Dataframes

In [10]:
import boto3
import sagemaker
import csv
import pandas as pd

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

## Get stored cleaned dataframes

In [11]:
%store -r
%who

bookings_grouped	 boto3	 bucket	 cancellations_grouped	 csv	 income_grouped	 income_value_grouped	 listings_clean	 listings_date	 
merge1	 pd	 region	 reviews_clean	 reviews_date	 role	 sagemaker	 sagemaker_session	 setup_dependencies_passed	 
setup_iam_roles_passed	 setup_instance_check_passed	 setup_s3_bucket_passed	 sm	 travelstats	 


In [12]:
reviews_date.head(5)

,Date,num_stays
0,2009-05-01,2
1,2009-06-01,2
2,2009-08-01,1
3,2009-09-01,1
4,2009-10-01,2


In [13]:
travelstats.head(5)

,Date,TotalAirlineTripstoDC,TotalAirlinePassengerstoDC,TotalAmericanTravelers,PercentofAmericanswhoTraveled,TotalTripsbyAmericans
0,2019-01-01,435,1326271.0,2.652241e+08,81.442421,1.137042e+09
1,2019-02-01,385,1261292.0,2.662408e+08,81.774326,1.148667e+09
2,2019-03-01,422,1668002.0,2.650919e+08,81.799085,1.306565e+09
3,2019-04-01,427,1681832.0,2.640626e+08,81.354347,1.446882e+09
4,2019-05-01,439,1766148.0,2.661340e+08,81.884670,1.415057e+09


In [14]:
bookings_grouped.head(5)

,Date,total_bookings
0,2014-10-01,322
1,2015-01-01,1576
2,2015-02-01,74
3,2015-03-01,149
4,2015-04-01,271


In [15]:
income_grouped.head(5)

,Date,income_by_year
0,1953-01-01,1
1,1954-01-01,1
2,1955-01-01,1
3,1956-01-01,1
4,1957-01-01,1


## Merge stored cleaned dataframes

In [33]:
merge1 = pd.merge(travelstats, reviews_date, on='Date')
merge1 = merge1.rename(columns={"numstays": "DCAirBnbStays"})

merge2 = pd.merge(income_grouped, bookings_grouped, on = 'Date')
merge2 = merge2.rename(columns={"income_by_year": "Income"})
merge2 = merge2.rename(columns={"total_bookings": "TotalBookings"})

merge3 = pd.merge(income_value_grouped, cancellations_grouped, on = 'Date')
merge3 = merge3.rename(columns={"income_total": "TotalIncome"})
merge3 = merge3.rename(columns={"cancellations": "TotalCancellations"})

merge4 = pd.merge(merge1, merge2, on='Date')
finaldf = pd.merge(merge3, merge4, on='Date')

merge1.head()

,Date,TotalAirlineTripstoDC,TotalAirlinePassengerstoDC,TotalAmericanTravelers,PercentofAmericanswhoTraveled,TotalTripsbyAmericans,num_stays
0,2019-01-01,435,1326271.0,2.652241e+08,81.442421,1.137042e+09,2899
1,2019-02-01,385,1261292.0,2.662408e+08,81.774326,1.148667e+09,2639
2,2019-03-01,422,1668002.0,2.650919e+08,81.799085,1.306565e+09,5226
3,2019-04-01,427,1681832.0,2.640626e+08,81.354347,1.446882e+09,5608
4,2019-05-01,439,1766148.0,2.661340e+08,81.884670,1.415057e+09,6110


In [23]:
%store finaldf

,Date,income_total,cancellations
0,2015-01-01,98224,1576
1,2016-01-01,101875,4160
2,2017-01-01,105734,4376


## Train, test, split

In [ ]:
test_size = 6

traindf = finaldf[:-test_size]
testdf = finaldf[-test_size:]

%store traindf
%store testdf